In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from pymongo import MongoClient
from pprint import pprint 



client = MongoClient('localhost', 27017)
db = client['crypto']
collection = db['btcusdt_price']

if (collection.count() != 0):
    collection.drop()

DATA = './data/BTCUSDT.csv'
df = pd.read_csv(DATA)
data = df.to_dict(orient='recoreds')

# inserting data to collection['btcusdt_price']
collection.insert_many(data)

# reading data from collection['btcusdt_price'] without id 
df_from_db = pd.DataFrame(list(collection.find({}, {'_id': 0})))
# df_from_db = pd.DataFrame(list(collection.find()))



## Experiments on CryptoQuant Data 

In [1]:
import requests
from pprint import pprint 
import configparser
from urllib.parse import urljoin 
from pprint import pprint 


config = configparser.ConfigParser()
config.read('config.ini')

API_KEY = config['CRYPTOQUANT']['API_KEY']
API_URL = config['CRYPTOQUANT']['API_URL']

headers = {'Authorization': 'Bearer ' + API_KEY}

### Inflow Data Generation

In [5]:
PATH = 'btc/exchange-flows/inflow'
URL = urljoin(API_URL, PATH)
params = {
    'exchange': 'all_exchange',
    'window': 'block',
    'limit': 100000,
    'to': 677958

}

response = requests.get(URL, headers=headers, params=params).json()
# print(response['status'])
inflow_all_exchange = response['result']['data']
len(inflow_all_exchange)



PATH = 'btc/exchange-flows/inflow'
URL = urljoin(API_URL, PATH)
params = {
    'exchange': 'binance',
    'window': 'block',
    'limit': 300000,
    'to': 677958

}

response = requests.get(URL, headers=headers, params=params).json()
# print(response['status'])
inflow_binance = response['result']['data']
len(inflow_binance)



PATH = 'btc/market-data/price-usd'
URL = urljoin(API_URL, PATH)
params = {
    'window': 'block',
    'limit': 100000,
    'to': 677958
}

response = requests.get(URL, headers=headers, params=params).json()
price = response['result']['data']
len(price)

100000

In [9]:
import pandas as pd
df_price = pd.DataFrame(price).sort_values(by=['blockheight'], ascending=True)
df_price = df_price.reset_index(drop=True)

# last two elements 
df_price[-2:] 
# df_price
# print(pd.isnull(df_price).sum())


,blockheight,datetime,price_usd_open,price_usd_high,price_usd_low,price_usd_close
99998,677957,2021-04-06 02:17:17,58766.046667,58804.258333,58759.705000,58802.185000
99999,677958,2021-04-06 02:18:50,58811.261667,58836.113333,58804.178333,58834.458333


In [10]:
df_inflow_all_exchange = pd.DataFrame(inflow_all_exchange).sort_values(by=['blockheight'],ascending=True)
df_inflow_all_exchange = df_inflow_all_exchange.reset_index(drop=True)
df_inflow_all_exchange[-2:]
# print(pd.isnull(df_inflow).sum())

,blockheight,datetime,inflow_total,inflow_top10,inflow_mean
99998,677957,2021-04-06 02:17:17,31.328702,31.287258,0.204763
99999,677958,2021-04-06 02:18:50,53.329586,51.451445,0.269341


In [12]:
df_price['blockheight'] == df_inflow_all_exchange['blockheight']
df_price['datetime'] == df_inflow_all_exchange['datetime']

0        True
1        True
2        True
3        True
4        True
         ... 
99995    True
99996    True
99997    True
99998    True
99999    True
Name: datetime, Length: 100000, dtype: bool

### Correlation between price and inflow_total

In [14]:
import numpy as np 

price_change = df_price['price_usd_close'].diff().fillna(0)
price_change

x= price_change
y = df_inflow_all_exchange['inflow_total']
print(np.corrcoef(x, y))

x= df_price['price_usd_close']
y = df_inflow_all_exchange['inflow_total']
print(np.corrcoef(x, y))

[[1.         0.01144677]
 [0.01144677 1.        ]]
[[1.         0.03095573]
 [0.03095573 1.        ]]


In [134]:
df_price

,blockheight,datetime,price_usd_open,price_usd_high,price_usd_low,price_usd_close
0,577959,2019-05-27 03:00:36,8731.622500,8745.612500,8730.677500,8745.612500
1,577960,2019-05-27 03:00:34,8731.622500,8745.612500,8730.677500,8745.612500
2,577961,2019-05-27 03:00:40,8731.622500,8745.612500,8730.677500,8745.612500
3,577962,2019-05-27 03:01:54,8742.700000,8754.722500,8742.042500,8750.815000
4,577963,2019-05-27 03:33:15,8744.140000,8746.212500,8737.575000,8738.080000
...,...,...,...,...,...,...
99995,677954,2021-04-06 02:03:56,58755.023333,58780.925000,58705.976667,58740.426667
99996,677955,2021-04-06 02:06:32,58729.781667,58800.028333,58718.980000,58771.081667
99997,677956,2021-04-06 02:16:47,58748.600000,58754.286667,58730.340000,58747.383333
99998,677957,2021-04-06 02:17:17,58766.046667,58804.258333,58759.705000,58802.185000


In [205]:
inflow_threshold = 1000
# filter indexes with inflow > 1000 
inflow_big = df_inflow_all_exchange.index[df_inflow_all_exchange['inflow_total'] >= inflow_threshold]


# def subset_sum(item, offset):
#     #  print('X item: ', item)
#     subsum = test_price['price_usd_close'].iloc[item.name+1: item.name+offset].sum() 
#     print(subsum)
#     print('item: ', item.name)
#     if (np.isnan(subsum)):
#         print('item: ', item.name)
#         print('price: ', item['price_usd_close'])
#     return(subsum)

def subset_sum(item, offset):
     return df_price['price_usd_close'].iloc[item.name+1: item.name+offset].mean() - item['price_usd_close']

# def subset_sum(start_index, offset):
#      return df_price['price_usd_close'].iloc[start_index+1: start_index+offset].sum()

def inflow_to_price_change(df_price, offset, inflow_big_indexes): 
    return df_price.apply(lambda x: subset_sum(x, offset) if (x.name in inflow_big_indexes) else 0 , axis=1)
    
# def inflow_to_price_change(df_price, offset, inflow_big_indexes): 
#    return df_price.apply(lambda x: subset_sum(x.name, offset) if (x.name in inflow_big_indexes) else 0 , axis=1)

offset = 10 
price_change = inflow_to_price_change(df_price, offset, inflow_big)

price_change
# price_change.iloc[32]



0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
99995    0.0
99996    0.0
99997    0.0
99998    0.0
99999    0.0
Length: 100000, dtype: float64

In [208]:
price_change
type(inflow_big)

df_price_change = pd.DataFrame(price_change.iloc[inflow_big], columns=['price_change']) 
df_price_change

,price_change
32,-1.265556
100,-10.936759
176,-42.904537
208,-20.747778
224,41.128426
...,...
99665,196.326852
99752,-167.305000
99861,-264.288519
99868,169.056296


In [236]:
df_inflow_big = df_inflow_all_exchange[df_inflow_all_exchange['inflow_total'] >= inflow_threshold]
df_inflow_big

df_price_change = pd.DataFrame(price_change.iloc[inflow_big], columns=['price_change']) 
df_price_change

,price_change
32,-1.265556
100,-10.936759
176,-42.904537
208,-20.747778
224,41.128426
...,...
99665,196.326852
99752,-167.305000
99861,-264.288519
99868,169.056296


In [237]:
df_inflow_big = df_inflow_big[:30]
df_price_change = df_price_change[:30]

In [238]:
import plotly.express as px
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig.add_trace(go.Scatter(x=df_inflow_big.datetime, y=df_price_change.price_change, mode='markers', name='price'), secondary_y=False)
# fig.add_trace(go.Scatter(x=df_inflow_big.datetime, y=df_inflow_big.inflow_total, mode='markers', name='inflow'), secondary_y=True)
fig.add_trace(go.Bar(x=df_inflow_big.datetime, y=df_price_change.price_change,  name='price'), secondary_y=False)
fig.add_trace(go.Bar(x=df_inflow_big.datetime, y=df_inflow_big.inflow_total, name='inflow'), secondary_y=True)
fig.update_layout(height=1000, width=1500)
fig.show()
# e2 = go.Scatter(x=df_price_and_inflow.datetime_x, y=df_price_and_inflow.inflow_total, mode='lines')
# |ip# lot(go.Figure([trace1, trace2]))


In [171]:
df = df_price.copy()

df['price_change'] = pd.DataFrame(price_change)
df
# df[['price_change', 'infflow_total']]  = pd.DataFrame(price_change), df_inflow_all_exchange['inflow_total']

# df1 = df_inflow_all_exchange['inflow_total']

,blockheight,datetime,price_usd_open,price_usd_high,price_usd_low,price_usd_close,price_change
0,577959,2019-05-27 03:00:36,8731.622500,8745.612500,8730.677500,8745.612500,0.0
1,577960,2019-05-27 03:00:34,8731.622500,8745.612500,8730.677500,8745.612500,0.0
2,577961,2019-05-27 03:00:40,8731.622500,8745.612500,8730.677500,8745.612500,0.0
3,577962,2019-05-27 03:01:54,8742.700000,8754.722500,8742.042500,8750.815000,0.0
4,577963,2019-05-27 03:33:15,8744.140000,8746.212500,8737.575000,8738.080000,0.0
...,...,...,...,...,...,...,...
99995,677954,2021-04-06 02:03:56,58755.023333,58780.925000,58705.976667,58740.426667,0.0
99996,677955,2021-04-06 02:06:32,58729.781667,58800.028333,58718.980000,58771.081667,0.0
99997,677956,2021-04-06 02:16:47,58748.600000,58754.286667,58730.340000,58747.383333,0.0
99998,677957,2021-04-06 02:17:17,58766.046667,58804.258333,58759.705000,58802.185000,0.0


In [161]:
df_inflow_all_exchange['inflow_total']

0         73.098934
1          0.727107
2          8.491730
3         82.828306
4        535.006831
            ...    
99995    338.229814
99996    519.873703
99997    625.737379
99998     31.328702
99999     53.329586
Name: inflow_total, Length: 100000, dtype: float64

In [157]:
df_price['price_change'].iloc[32]

78562.03

In [120]:
df_price_and_inflow_all_exchange = pd.merge(df_price, df_inflow_all_exchange, on='blockheight')
# df_price_and_inflow
df_price_and_inflow_all_exchange[['datetime_x', 'price_usd_close', 'inflow_total']]




,datetime_x,price_usd_close,inflow_total
0,2019-05-27 03:00:36,8745.612500,73.098934
1,2019-05-27 03:00:34,8745.612500,0.727107
2,2019-05-27 03:00:40,8745.612500,8.491730
3,2019-05-27 03:01:54,8750.815000,82.828306
4,2019-05-27 03:33:15,8738.080000,535.006831
...,...,...,...
99995,2021-04-06 02:03:56,58740.426667,338.229814
99996,2021-04-06 02:06:32,58771.081667,519.873703
99997,2021-04-06 02:16:47,58747.383333,625.737379
99998,2021-04-06 02:17:17,58802.185000,31.328702


In [127]:
df_data = df_price_and_inflow_all_exchange[-500:]
df_data

,blockheight,datetime_x,price_usd_open,price_usd_high,price_usd_low,price_usd_close,datetime_y,inflow_total,inflow_top10,inflow_mean
99500,677459,2021-04-02 14:35:51,59094.423333,59127.626667,59087.711667,59108.346667,2021-04-02 14:35:51,55.270356,54.696041,0.327044
99501,677460,2021-04-02 14:36:10,59104.351667,59110.121667,59074.571667,59087.520000,2021-04-02 14:36:10,89.843575,89.439502,0.485641
99502,677461,2021-04-02 14:37:43,59096.986667,59136.686667,59088.166667,59120.836667,2021-04-02 14:37:43,27.712902,27.042992,0.148994
99503,677462,2021-04-02 14:37:56,59096.986667,59136.686667,59088.166667,59120.836667,2021-04-02 14:37:56,16.462023,16.298038,0.126631
99504,677463,2021-04-02 14:45:02,59300.853333,59320.946667,59279.458333,59289.063333,2021-04-02 14:45:02,129.751237,127.377511,0.690166
...,...,...,...,...,...,...,...,...,...,...
99995,677954,2021-04-06 02:03:56,58755.023333,58780.925000,58705.976667,58740.426667,2021-04-06 02:03:56,338.229814,336.425901,1.363830
99996,677955,2021-04-06 02:06:32,58729.781667,58800.028333,58718.980000,58771.081667,2021-04-06 02:06:32,519.873703,479.883041,2.487434
99997,677956,2021-04-06 02:16:47,58748.600000,58754.286667,58730.340000,58747.383333,2021-04-06 02:16:47,625.737379,450.932535,1.835007
99998,677957,2021-04-06 02:17:17,58766.046667,58804.258333,58759.705000,58802.185000,2021-04-06 02:17:17,31.328702,31.287258,0.204763


In [105]:
from sklearn.preprocessing import StandardScaler

df_data[['norm_inflow']] = StandardScaler().fit_transform(df_data[['inflow_total']])
df_data['norm_price'] = df_data['price_usd_close']/10000
# df_data[['norm_price', 'norm_inflow']] = StandardScaler().fit_transform(df_data[['price_usd_close', 'inflow_total']])


<ipython-input-105-89d0313a3526>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/hjchoi/dev/crypto/.venv/lib/python3.8/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-105-89d0313a3526>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [128]:
df_data

,blockheight,datetime_x,price_usd_open,price_usd_high,price_usd_low,price_usd_close,datetime_y,inflow_total,inflow_top10,inflow_mean
99500,677459,2021-04-02 14:35:51,59094.423333,59127.626667,59087.711667,59108.346667,2021-04-02 14:35:51,55.270356,54.696041,0.327044
99501,677460,2021-04-02 14:36:10,59104.351667,59110.121667,59074.571667,59087.520000,2021-04-02 14:36:10,89.843575,89.439502,0.485641
99502,677461,2021-04-02 14:37:43,59096.986667,59136.686667,59088.166667,59120.836667,2021-04-02 14:37:43,27.712902,27.042992,0.148994
99503,677462,2021-04-02 14:37:56,59096.986667,59136.686667,59088.166667,59120.836667,2021-04-02 14:37:56,16.462023,16.298038,0.126631
99504,677463,2021-04-02 14:45:02,59300.853333,59320.946667,59279.458333,59289.063333,2021-04-02 14:45:02,129.751237,127.377511,0.690166
...,...,...,...,...,...,...,...,...,...,...
99995,677954,2021-04-06 02:03:56,58755.023333,58780.925000,58705.976667,58740.426667,2021-04-06 02:03:56,338.229814,336.425901,1.363830
99996,677955,2021-04-06 02:06:32,58729.781667,58800.028333,58718.980000,58771.081667,2021-04-06 02:06:32,519.873703,479.883041,2.487434
99997,677956,2021-04-06 02:16:47,58748.600000,58754.286667,58730.340000,58747.383333,2021-04-06 02:16:47,625.737379,450.932535,1.835007
99998,677957,2021-04-06 02:17:17,58766.046667,58804.258333,58759.705000,58802.185000,2021-04-06 02:17:17,31.328702,31.287258,0.204763


In [133]:
price_change = df_data['price_usd_close'].diff().fillna(0)
price_change


99500      0.000000
99501    -20.826667
99502     33.316667
99503      0.000000
99504    168.226667
            ...    
99995   -341.051667
99996     30.655000
99997    -23.698333
99998     54.801667
99999     32.273333
Name: price_usd_close, Length: 500, dtype: float64

In [131]:
df_data

,blockheight,datetime_x,price_usd_open,price_usd_high,price_usd_low,price_usd_close,datetime_y,inflow_total,inflow_top10,inflow_mean
99500,677459,2021-04-02 14:35:51,59094.423333,59127.626667,59087.711667,59108.346667,2021-04-02 14:35:51,55.270356,54.696041,0.327044
99501,677460,2021-04-02 14:36:10,59104.351667,59110.121667,59074.571667,59087.520000,2021-04-02 14:36:10,89.843575,89.439502,0.485641
99502,677461,2021-04-02 14:37:43,59096.986667,59136.686667,59088.166667,59120.836667,2021-04-02 14:37:43,27.712902,27.042992,0.148994
99503,677462,2021-04-02 14:37:56,59096.986667,59136.686667,59088.166667,59120.836667,2021-04-02 14:37:56,16.462023,16.298038,0.126631
99504,677463,2021-04-02 14:45:02,59300.853333,59320.946667,59279.458333,59289.063333,2021-04-02 14:45:02,129.751237,127.377511,0.690166
...,...,...,...,...,...,...,...,...,...,...
99995,677954,2021-04-06 02:03:56,58755.023333,58780.925000,58705.976667,58740.426667,2021-04-06 02:03:56,338.229814,336.425901,1.363830
99996,677955,2021-04-06 02:06:32,58729.781667,58800.028333,58718.980000,58771.081667,2021-04-06 02:06:32,519.873703,479.883041,2.487434
99997,677956,2021-04-06 02:16:47,58748.600000,58754.286667,58730.340000,58747.383333,2021-04-06 02:16:47,625.737379,450.932535,1.835007
99998,677957,2021-04-06 02:17:17,58766.046667,58804.258333,58759.705000,58802.185000,2021-04-06 02:17:17,31.328702,31.287258,0.204763


## Test: Inflow changes Price?

In [134]:
import plotly.express as px
import plotly.graph_objects as go 


from plotly.offline import init_notebook_mode, iplot

from plotly.subplots import make_subplots



fig = make_subplots(specs=[[{"secondary_y": True}]])# 
afig.add_trace(go.Scatter(x=df_data.datetime_x, y=price_change, name='price'), secondary_y=Falsee
1fig.add_trace(go.Scatter(x=df_data.datetime_x, y=df_data.inflow_total, name='inflow'), secondary_y=True)
ffig.update_layout(height=1000, width=1500)ig.show()e2 = go.Scatter(x=df_price_and_inflow.datetime_x, y=df_price_and_inflow.inflow_total, mode='lines')
ip# lot(go.Figure([trace1, trace2]))

ig =o.Fi#
 gure()a
fig = px.li# ne(df_price_df_datatime_x', y=['price_usdnorm_pricew_tonorm_inflow='Prheight=1000, width=1500w 

# # fig = px.line(df_data, x='datetime_x', y=['price_usd_close', 'inflow_total'], height=1000, width=1500) 
r
#fi# g = make_subplots(specs=[[{'secondary_y': True}]])
f
fig# fig = pline(df_price_and_inflow, x='datetime_x', y='price_usd_close', title='Price')
a# dd_scatter(cq_idf_price_and_inflow'datetime', y_x='inflow_total', title='Inflo')ea_c


h
fig =#  px._price_and_inflow='datetime', _xy='infprice_usd_closetitle='InfPrice Changear
earea_char# t = px.area(df_price_and_inflow, x='datetime_x', y='inflow_total', title='Price Change')_chart.upda# te_xaxes(title_text = 'Date')
area_chart.updat# e_yaxes(title_text = 'Infl'Price Change'kprefix = '$')
area_chart.update# _layout(showlegend = False)
area_chart.show()# 

In [112]:
df_data['price_usd_close'].diff()

type(df_data['price_usd_close'].diff())

pandas.core.series.Series

In [136]:
import numpy as np 

x= price_change
y = df_data['inflow_total']
print(np.corrcoef(x, y))


x= df_data['price_usd_close']
y = df_data['inflow_total']
print(np.corrcoef(x, y))

[[ 1.         -0.00431346]
 [-0.00431346  1.        ]]
[[1.         0.08276465]
 [0.08276465 1.        ]]


In [95]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=[1, 2, 3], y=[40, 50, 60], name="yaxis data"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=[2, 3, 4], y=[4, 5, 6], name="yaxis2 data"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Double Y Axis Example"
)

# Set x-axis title
fig.update_xaxes(title_text="xaxis title")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

fig.show()

In [16]:
#fig = make_subplots(specs=[[{'secondary_y': True}]])
fig = go.Figure()

fig = px.line(cq_price, x='datetime', y=['price_usd_close', 'inflow_total'], title='Price')
# fig.add_scatter(cq_inflow, x='datetime', y='inflow_total', title='Inflo')


# fig.add_trace(go.Scatter(x=cq_price['datetime'], y=cq_prce['price_usd_close']),  marker=dict(color="blue"),showlegend=False)
# fig.add_trace(go.Scatter(x=cq_price['datetime'], y=cq_inflow['inflow_total']),  marker=dict(color="green"),showlegend=False)

fig.show()


ValueError: All arguments should have the same length. The length of argument `y` is 2, whereas the length of  previously-processed arguments ['datetime'] is 100000

In [15]:
trace1 = go.Scatter(x=df_price_and_inflow.datetime_x, y=df_price_and_inflow.price_usd_close, mode='lines')
trace2 = go.Scatter(x=df_price_and_inflow.datetime_x, y=df_price_and_inflow.inflow_total, mode='lines')
iplot(go.Figure([trace1, trace2]))

NameError: name 'iplot' is not defined

In [ ]:
cq_data[0:5]

In [ ]:
PATH = 'btc/exchange-flows/reserve'
URL = urljoin(API_URL, PATH)
params = {
    'exchange': 'binance',
    'window': 'block',
    'limit': 100000
}

response = requests.get(URL, headers=headers, params=params).json()
# print(response['status'])
cq_data = response['result']['data']
len(cq_data)

In [ ]:
cq_data[0:5]

In [ ]:
PATH = 'btc/exchange-flows/outflow'
URL = urljoin(API_URL, PATH)
params = {
    'exchange': 'binance',
    'window': 'block',
    'limit': 100000
}

response = requests.get(URL, headers=headers, params=params).json()
# print(response['status'])
cq_data = response['result']['data']
len(cq_data)

In [ ]:
df_cq_data = pd.DataFrame(cq_data)

In [ ]:
# df_cq_data['datetime']
df_cq_data[0:5]

In [ ]:
import plotly.express as px
import plotly.graph_objects as go 

area_chart = px.area(df_cq_data, x='datetime', y='inflow_total', title='Inflow Total')
area_chart.update_xaxes(title_text = 'Date')
area_chart.update_yaxes(title_text = 'Inflow Total', tickprefix = '$')
area_chart.update_layout(showlegend = False)
area_chart.show()



In [ ]:
candlestick = go.Figure(data = [go.Candlestick(x = df_from_db['Date'], 
                                               open = df_from_db['Open'], 
                                               high = df_from_db['High'], 
                                               low = df_from_db['Low'], 
                                               close = df_from_db['Close'])])

candlestick.update_layout(xaxis_rangeslider_visible = False, title = 'BTCUSDT')
candlestick.update_xaxes(title_text = 'Date')
candlestick.update_yaxes(title_text = 'BTCUSDT Close Price', tickprefix = '$')

candlestick.show()

In [ ]:
df['Close']

In [ ]:
df['Close'].plot()

In [ ]:
max(df['Close'])

In [ ]:
min(df['Close'])

In [ ]:
df['Close'].rolling(3).mean()



In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

ax.set_title('INDEX', fontsize=15)
ax.set_ylabel("BTCUSDT")
ax.set_xlabel("Date Time")
ax.plot(df.index, df[['Close'], df['Close'].rolling(3).mean])
ax.legend(['Close','MA5','MA10'])
plt.show()